# Berechnung der Matrixeinträge für den Übergang mit der Fictitious-Node-Methode

In [1]:
import sympy as sp
from IPython.display import display

# Definition der Symbole für Diffusionskoeffizienten und Variablen
D_1, D_2 = sp.symbols('D_1 D_2')  # Diffusionskoeffizienten D_1 und D_2 für zwei verschiedene Schichten
a_i_m, a_i, a_ip_dx_2, a_ip_dx, b_i_p, b_i, b_im_dx_2, b_im_dx = sp.symbols('a_{i-1} a_{i} a_{i+dx/2} a_{i+dx} b_{i+1} b_{i} b_{i-dx/2} b_{i-dx}')
dx, dt = sp.symbols('\Delta_x \Delta_t')  # Schrittweite dx und Zeitschritt dt
alpha_1, alpha_2 = sp.symbols('alpha_1 alpha_2')  # Skalierungsfaktoren für die Diskretisierung

# Berechnungsterm für die Diffusion (alphan abhängig von Diffusionskoeffizienten, dx und dt)
alpha_1_term = (D_1 * dt ) / (dx**2)
alpha_2_term = (D_2 * dt ) / (dx**2)

# Definition der Flusskontinuitätsgleichungen
# Links: Fluss auf der Seite von Material A (abhängig von D_1, a_ip_dx und a_i_m)
lhs_flux_continuity = D_1 * (a_ip_dx - a_i_m) / (dx)
# Rechts: Fluss auf der Seite von Material B (abhängig von D_2, b_i_p und b_im_dx)
rhs_flux_continuity = D_2 * (b_i_p - b_im_dx) / (dx)
# Setze die beiden Flusskontinuitäten gleich (Kontinuitätsgleichung für den Fluss)
flux_continuity = sp.Eq(lhs_flux_continuity, rhs_flux_continuity)
display(flux_continuity)  # Anzeige der Flusskontinuitätsgleichung

# Löse die Flusskontinuitätsgleichung nach a_ip_dx auf (Umformung der Gleichung)
a_ip_dx_rew = sp.solve(flux_continuity, a_ip_dx)[0]
display(sp.Eq(a_ip_dx, a_ip_dx_rew))  # Zeige die umgeformte Gleichung an

# Definition der Konzentrationskontinuitätsgleichungen (an der Grenzfläche)
# Links: Mittlere Konzentration auf der A-Seite
lhs_concentration_continuity = (a_ip_dx + a_i) / 2
# Rechts: Mittlere Konzentration auf der B-Seite
rhs_concentration_continuity = (b_i + b_im_dx) / 2
# Setze die Konzentrationen gleich (Kontinuitätsgleichung für Konzentrationen)
concentration_continuity = sp.Eq(lhs_concentration_continuity, rhs_concentration_continuity)
display(concentration_continuity)

# Substituiere die umgeformte a_ip_dx Gleichung in die Konzentrationskontinuitätsgleichung
concentration_continuity_sub = concentration_continuity.subs(a_ip_dx, a_ip_dx_rew)
# Löse nach b_im_dx auf und vereinfache das Ergebnis
b_im_dx_rew = sp.solve(concentration_continuity_sub, b_im_dx)[0].simplify()
display(sp.Eq(b_im_dx, b_im_dx_rew))  # Zeige das Ergebnis für b_im_dx an

# Substituiere b_im_dx in die umgeformte Gleichung für a_ip_dx
a_ip_dx_final = a_ip_dx_rew.subs(b_im_dx, b_im_dx_rew).simplify()
display(sp.Eq(a_ip_dx, a_ip_dx_final))  # Zeige das finale Ergebnis für a_ip_dx an

# Substituiere a_ip_dx_final in die ursprüngliche Flusskontinuitätsgleichung
flux_continuity_sub = flux_continuity.subs(a_ip_dx, a_ip_dx_final)
display(flux_continuity_sub)  # Zeige die modifizierte Flusskontinuität an

# Löse die modifizierte Flusskontinuitätsgleichung nach b_im_dx
solutions = sp.solve(flux_continuity_sub, b_im_dx)
display(solutions)  # Zeige die Lösungen für b_im_dx an

# Schicht 1 (Layer 1) – Diskretisierung der Diffusionsgleichung für Material A
# Linke Seite der Diskretisierungsgleichung (LHS)
a_lhs = - alpha_1_term * a_i_m + (1 + 2 * alpha_1_term) * a_i - alpha_1_term * a_ip_dx
# Rechte Seite der Diskretisierungsgleichung (RHS)
a_rhs =  alpha_1_term * a_i_m + (1 - 2 * alpha_1_term) * a_i + alpha_1_term * a_ip_dx

# Gleichung für die Diskretisierung von Schicht 1
a_eq = sp.Eq(a_lhs, a_rhs)
# Substituiere die finale a_ip_dx Gleichung in die Diskretisierungsgleichung von Schicht 1
a_eq_subs_a = a_eq.subs(a_ip_dx, a_ip_dx_final)
# Substituiere den Wert für alpha_1_term und expandiere die Gleichung
a_eq_subs_alpha = a_eq_subs_a.subs(alpha_1_term, alpha_1).expand()
display(a_eq_subs_alpha)  # Zeige die resultierende Gleichung an

lhs_expr_a = a_eq_subs_alpha.lhs  # Linke Seite der Gleichung (LHS)
rhs_expr_a = a_eq_subs_alpha.rhs  # Rechte Seite der Gleichung (RHS)

# Bestimme die Koeffizienten von a_i, a_i_m und b_i auf der linken Seite (LHS)
coeff_a_i_lhs = lhs_expr_a.coeff(a_i)
coeff_a_im_lhs = lhs_expr_a.coeff(a_i_m)
coeff_b_i_lhs = lhs_expr_a.coeff(b_i)

# Bestimme die Koeffizienten von a_i, a_i_m und b_i auf der rechten Seite (RHS)
coeff_a_i_rhs = rhs_expr_a.coeff(a_i)
coeff_a_im_rhs = rhs_expr_a.coeff(a_i_m)
coeff_b_i_rhs = rhs_expr_a.coeff(b_i)

# Rekonstruiere die linke Seite (LHS) unter Verwendung der extrahierten Koeffizienten
lhs_reconstructed = coeff_a_i_lhs * a_i + coeff_a_im_lhs * a_i_m + coeff_b_i_lhs * b_i
# Rekonstruiere die rechte Seite (RHS) unter Verwendung der extrahierten Koeffizienten
rhs_reconstructed = coeff_a_i_rhs * a_i + coeff_a_im_rhs * a_i_m + coeff_b_i_rhs * b_i

# Erstelle eine neue Gleichung mit der rekonstruierten linken und rechten Seite
new_eq_a = sp.Eq(lhs_reconstructed, rhs_reconstructed)
display(new_eq_a)  # Zeige die neue Gleichung für Schicht 1 an

# Schicht 2 (Layer 2) – Diskretisierung der Diffusionsgleichung für Material B
# Linke Seite der Diskretisierungsgleichung (LHS)
b_lhs = - alpha_2_term * b_im_dx + (1 + 2 * alpha_2_term) * b_i - alpha_2_term * b_i_p
# Rechte Seite der Diskretisierungsgleichung (RHS)
b_rhs =  alpha_2_term * b_im_dx + (1 - 2 * alpha_2_term) * b_i + alpha_2_term * b_i_p

# Gleichung für die Diskretisierung von Schicht 2
b_eq = sp.Eq(b_lhs, b_rhs)
# Substituiere b_im_dx in die Diskretisierungsgleichung von Schicht 2
b_eq_subs_b = b_eq.subs(b_im_dx, b_im_dx_rew)
# Substituiere den Wert für alpha_2_term und expandiere die Gleichung
b_eq_subs_alpha = b_eq_subs_b.subs(alpha_2_term, alpha_2).expand()

lhs_expr_b = b_eq_subs_alpha.lhs  # Linke Seite der Gleichung (LHS)
rhs_expr_b = b_eq_subs_alpha.rhs  # Rechte Seite der Gleichung (RHS)

# Bestimme die Koeffizienten von b_i, b_i_p und a_i auf der linken Seite (LHS)
coeff_b_i_lhs = lhs_expr_b.coeff(b_i)
coeff_b_ip_lhs = lhs_expr_b.coeff(b_i_p)
coeff_a_ib_lhs = lhs_expr_b.coeff(a_i)

# Bestimme die Koeffizienten von b_i, b_i_p und a_i auf der rechten Seite (RHS)
coeff_b_i_rhs = rhs_expr_b.coeff(b_i)
coeff_b_ip_rhs = rhs_expr_b.coeff(b_i_p)
coeff_a_ib_rhs = rhs_expr_b.coeff(a_i)

# Rekonstruiere die linke Seite (LHS) unter Verwendung der extrahierten Koeffizienten
lhs_reconstructed_b = coeff_b_i_lhs * b_i + coeff_a_ib_lhs * a_i + coeff_b_ip_lhs * b_i_p
# Rekonstruiere die rechte Seite (RHS) unter Verwendung der extrahierten Koeffizienten
rhs_reconstructed_b = coeff_b_i_rhs * b_i + coeff_a_ib_rhs * a_i + coeff_b_ip_rhs * b_i_p

# Erstelle eine neue Gleichung mit der rekonstruierten linken und rechten Seite
new_eq_b = sp.Eq(lhs_reconstructed_b, rhs_reconstructed_b)
display(new_eq_b)  # Zeige die neue Gleichung für Schicht 2 an


Eq(D_1*(a_{i+dx} - a_{i-1})/\Delta_x, D_2*(b_{i+1} - b_{i-dx})/\Delta_x)

Eq(a_{i+dx}, (D_1*a_{i-1} + D_2*b_{i+1} - D_2*b_{i-dx})/D_1)

Eq(a_{i+dx}/2 + a_{i}/2, b_{i-dx}/2 + b_{i}/2)

Eq(b_{i-dx}, (D_1*a_{i-1} + D_1*a_{i} - D_1*b_{i} + D_2*b_{i+1})/(D_1 + D_2))

Eq(a_{i+dx}, (D_1*a_{i-1} - D_2*a_{i} + D_2*b_{i+1} + D_2*b_{i})/(D_1 + D_2))

Eq(D_1*(-a_{i-1} + (D_1*a_{i-1} - D_2*a_{i} + D_2*b_{i+1} + D_2*b_{i})/(D_1 + D_2))/\Delta_x, D_2*(b_{i+1} - b_{i-dx})/\Delta_x)

[(D_1*a_{i-1} + D_1*a_{i} - D_1*b_{i} + D_2*b_{i+1})/(D_1 + D_2)]

Eq(-D_1*a_{i-1}*alpha_1/(D_1 + D_2) + D_2*a_{i}*alpha_1/(D_1 + D_2) - D_2*alpha_1*b_{i+1}/(D_1 + D_2) - D_2*alpha_1*b_{i}/(D_1 + D_2) - a_{i-1}*alpha_1 + 2*a_{i}*alpha_1 + a_{i}, D_1*a_{i-1}*alpha_1/(D_1 + D_2) - D_2*a_{i}*alpha_1/(D_1 + D_2) + D_2*alpha_1*b_{i+1}/(D_1 + D_2) + D_2*alpha_1*b_{i}/(D_1 + D_2) + a_{i-1}*alpha_1 - 2*a_{i}*alpha_1 + a_{i})

Eq(-D_2*alpha_1*b_{i}/(D_1 + D_2) + a_{i-1}*(-D_1*alpha_1/(D_1 + D_2) - alpha_1) + a_{i}*(D_2*alpha_1/(D_1 + D_2) + 2*alpha_1 + 1), D_2*alpha_1*b_{i}/(D_1 + D_2) + a_{i-1}*(D_1*alpha_1/(D_1 + D_2) + alpha_1) + a_{i}*(-D_2*alpha_1/(D_1 + D_2) - 2*alpha_1 + 1))

Eq(-D_1*a_{i}*alpha_2/(D_1 + D_2) + b_{i+1}*(-D_2*alpha_2/(D_1 + D_2) - alpha_2) + b_{i}*(D_1*alpha_2/(D_1 + D_2) + 2*alpha_2 + 1), D_1*a_{i}*alpha_2/(D_1 + D_2) + b_{i+1}*(D_2*alpha_2/(D_1 + D_2) + alpha_2) + b_{i}*(-D_1*alpha_2/(D_1 + D_2) - 2*alpha_2 + 1))

# Substituieren für die vereinfachte Darstellung

In [2]:
# Definiere die Symbole phi und theta
phi, theta = sp.symbols('phi theta')

# Berechne die Terme theta und phi als Brüche der Diffusionskoeffizienten
theta_term = D_1 / (D_1 + D_2)  # Verhältnis der Diffusionskoeffizienten von Schicht A zu A+B
phi_term = D_2 / (D_1 + D_2)    # Verhältnis der Diffusionskoeffizienten von Schicht B zu A+B

# SCHICHT A (Layer A)
# Substituiere die Terme phi und theta in die Gleichung für Schicht A ein und expandiere die Gleichung
new_eq_a_subs = new_eq_a.subs({phi_term: phi, theta_term: theta}).expand()
display(new_eq_a_subs)  # Zeige die neue Gleichung für Schicht A an

# Extrahiere die linke und rechte Seite der neuen Gleichung
lhs_expr = new_eq_a_subs.lhs  # Linke Seite der Gleichung
rhs_expr = new_eq_a_subs.rhs  # Rechte Seite der Gleichung

# Extrahiere die Koeffizienten für die Variablen a_i, a_i_m und b_i aus der linken Seite (LHS) der Gleichung
coeff_a_i_lhs = lhs_expr.coeff(a_i)
coeff_a_im_lhs = lhs_expr.coeff(a_i_m)
coeff_b_i_lhs = lhs_expr.coeff(b_i)

# Extrahiere die Koeffizienten für die Variablen a_i, a_i_m und b_i aus der rechten Seite (RHS) der Gleichung
coeff_a_i_rhs = rhs_expr.coeff(a_i)
coeff_a_im_rhs = rhs_expr.coeff(a_i_m)
coeff_b_i_rhs = rhs_expr.coeff(b_i)

# Rekonstruiere die linke Seite (LHS), indem die Koeffizienten mit den Variablen multipliziert werden
lhs_recombined = coeff_a_i_lhs * a_i + coeff_a_im_lhs * a_i_m + coeff_b_i_lhs * b_i

# Rekonstruiere die rechte Seite (RHS), indem die Koeffizienten mit den Variablen multipliziert werden
rhs_recombined = coeff_a_i_rhs * a_i + coeff_a_im_rhs * a_i_m + coeff_b_i_rhs * b_i

# Ausgabe der rekonstruierten linken und rechten Seite der Gleichung für Schicht A
print('Linke Seite Schicht 1')
display(lhs_recombined)

print('Rechte Seite Schicht 1')
display(rhs_recombined)

# Erstelle die neue Gleichung, indem die rekonstruierten linken und rechten Seiten gleichgesetzt werden
recombined_eq = sp.Eq(lhs_recombined, rhs_recombined)

# SCHICHT B (Layer B)
# Substituiere die Terme phi und theta in die Gleichung für Schicht B ein und expandiere die Gleichung
new_eq_b_subs = new_eq_b.subs({phi_term: phi, theta_term: theta}).expand()

# Extrahiere die linke und rechte Seite der neuen Gleichung für Schicht B
lhs_expr_b_subs = new_eq_b_subs.lhs  # Linke Seite der Gleichung für Schicht B
rhs_expr_b_subs = new_eq_b_subs.rhs  # Rechte Seite der Gleichung für Schicht B

# Extrahiere die Koeffizienten für die Variablen b_i, b_i_p und a_i aus der linken Seite (LHS) der Gleichung für Schicht B
coeff_b_i_lhs_subs = lhs_expr_b_subs.coeff(b_i)
coeff_b_ip_lhs_subs = lhs_expr_b_subs.coeff(b_i_p)
coeff_a_ib_lhs_subs = lhs_expr_b_subs.coeff(a_i)

# Extrahiere die Koeffizienten für die Variablen b_i, b_i_p und a_i aus der rechten Seite (RHS) der Gleichung für Schicht B
coeff_b_i_rhs_subs = rhs_expr_b_subs.coeff(b_i)
coeff_b_ip_rhs_subs = rhs_expr_b_subs.coeff(b_i_p)
coeff_a_ib_rhs_subs = rhs_expr_b_subs.coeff(a_i)

# Rekonstruiere die linke Seite (LHS) der Gleichung für Schicht B, indem die Koeffizienten mit den Variablen multipliziert werden
lhs_recombined_b_subs = coeff_b_i_lhs_subs * b_i + coeff_b_ip_lhs_subs * b_i_p + coeff_a_ib_lhs_subs * a_i

# Rekonstruiere die rechte Seite (RHS) der Gleichung für Schicht B, indem die Koeffizienten mit den Variablen multipliziert werden
rhs_recombined_b_subs = coeff_b_i_rhs_subs * b_i + coeff_b_ip_rhs_subs * b_i_p + coeff_a_ib_rhs_subs * a_i

# Ausgabe der rekonstruierten linken und rechten Seite der Gleichung für Schicht B
print('Linke Seite Schicht 2')
display(lhs_recombined_b_subs)

print('Rechte Seite Schicht 2')
display(rhs_recombined_b_subs)

# Erstelle die neue Gleichung für Schicht B, indem die rekonstruierten linken und rechten Seiten gleichgesetzt werden
recombined_eq_b_subs = sp.Eq(lhs_recombined_b_subs, rhs_recombined_b_subs)


Eq(-a_{i-1}*alpha_1*theta - a_{i-1}*alpha_1 + a_{i}*alpha_1*phi + 2*a_{i}*alpha_1 + a_{i} - alpha_1*b_{i}*phi, a_{i-1}*alpha_1*theta + a_{i-1}*alpha_1 - a_{i}*alpha_1*phi - 2*a_{i}*alpha_1 + a_{i} + alpha_1*b_{i}*phi)

Linke Seite Schicht 1


a_{i-1}*(-alpha_1*theta - alpha_1) + a_{i}*(alpha_1*phi + 2*alpha_1 + 1) - alpha_1*b_{i}*phi

Rechte Seite Schicht 1


a_{i-1}*(alpha_1*theta + alpha_1) + a_{i}*(-alpha_1*phi - 2*alpha_1 + 1) + alpha_1*b_{i}*phi

Linke Seite Schicht 2


-a_{i}*alpha_2*theta + b_{i+1}*(-alpha_2*phi - alpha_2) + b_{i}*(alpha_2*theta + 2*alpha_2 + 1)

Rechte Seite Schicht 2


a_{i}*alpha_2*theta + b_{i+1}*(alpha_2*phi + alpha_2) + b_{i}*(-alpha_2*theta - 2*alpha_2 + 1)